**Importing libraries and packages**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile

# Specify the path to your ZIP file in Google Drive
zip_file_path = '/content/drive/MyDrive/cities.zip'

# Specify the destination directory where you want to extract the contents
extracted_path = '/content/dataset/'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import torchvision.models as models

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

**Loading dataset, splitting it and setting DataLoaders**

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ImageFolder(root='dataset/cities', transform=transform)
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [len(dataset) - 2 * int(len(dataset) * 0.15) , int(len(dataset) * 0.15), int(len(dataset) * 0.15)]
)
n_classes = len(dataset.classes)

In [6]:
n_classes

5

In [7]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

**Loading ResNet50**

In [8]:
model = models.resnet50(weights='ResNet50_Weights.DEFAULT')

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 90.4MB/s]


**Changing the last layer and freezing the model except this last layer**

In [9]:
model.fc=nn.Sequential(
    nn.Linear(in_features=model.fc.in_features, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=n_classes),
    nn.Softmax(dim=1)
)

In [10]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
for param in model.parameters():
    param.requires_grad = False

# Make the last layer trainable
for param in model.fc.parameters():
    param.requires_grad = True

**Training Loop**

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
num_epochs = 10
model.to(device)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)


    model.eval()
    val_loss = 0.0
    val_pred = [[] for i in range(5)]
    real = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            top_5_probas, top_5_indices = torch.topk(outputs, k=5, dim=1)
            for i in range(5):
                val_pred[i].extend(top_5_indices[:,i].cpu().numpy())
            real.extend(labels.cpu().numpy())
            break

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = [accuracy_score(real, val_pred[i]) for i in range(5)]
    for i in range(1,5):
        val_accuracy[i] += val_accuracy[i-1]

    print(f"Epoch [{epoch + 1}/{num_epochs}] - "
          f"Train Loss: {avg_loss:.4f}, "
          f"Validation Loss: {avg_val_loss:.4f}, \n")
    for i in range(5):
        print(f"Validation Accuracy Top{i+1}: {100*val_accuracy[i]:.2f}%")


Epoch [1/10] - Train Loss: 1.2090, Validation Loss: 0.0018, 

Validation Accuracy Top1: 78.12%
Validation Accuracy Top2: 90.62%
Validation Accuracy Top3: 96.88%
Validation Accuracy Top4: 100.00%
Validation Accuracy Top5: 100.00%
Epoch [2/10] - Train Loss: 1.1616, Validation Loss: 0.0017, 

Validation Accuracy Top1: 84.38%
Validation Accuracy Top2: 90.62%
Validation Accuracy Top3: 96.88%
Validation Accuracy Top4: 100.00%
Validation Accuracy Top5: 100.00%
Epoch [3/10] - Train Loss: 1.1415, Validation Loss: 0.0017, 

Validation Accuracy Top1: 81.25%
Validation Accuracy Top2: 90.62%
Validation Accuracy Top3: 96.88%
Validation Accuracy Top4: 100.00%
Validation Accuracy Top5: 100.00%
Epoch [4/10] - Train Loss: 1.1286, Validation Loss: 0.0015, 

Validation Accuracy Top1: 96.88%
Validation Accuracy Top2: 96.88%
Validation Accuracy Top3: 96.88%
Validation Accuracy Top4: 100.00%
Validation Accuracy Top5: 100.00%
Epoch [5/10] - Train Loss: 1.1174, Validation Loss: 0.0019, 

Validation Accuracy To

In [14]:
torch.save(model.state_dict(), 'model_state_dict.pth')
model.eval()
test_pred = [[] for i in range(5)]
real = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        top_5_probas, top_5_indices = torch.topk(outputs, k=5, dim=1)
        for i in range(5):
            test_pred[i].extend(top_5_indices[:,i].cpu().numpy())
        real.extend(labels.cpu().numpy())

test_accuracy = [accuracy_score(real, test_pred[i]) for i in range(5)]
for i in range(1,5):
        test_accuracy[i] += test_accuracy[i-1]

for i in range(5):
    print(f"Test Accuracy Top{i+1}: {100*test_accuracy[i]:.2f}%")


Test Accuracy Top1: 79.48%
Test Accuracy Top2: 92.48%
Test Accuracy Top3: 97.41%
Test Accuracy Top4: 99.30%
Test Accuracy Top5: 100.00%
